In [4]:
import lyricsgenius
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re #regular expressions
%matplotlib inline
import math
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Initiate packages to interact with APIs

In [3]:
#login for spotipy api/package
id_ = "insertyouridhere"
secret = "insertyoursecrethere"
ccm = SpotifyClientCredentials(client_id=id_, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=ccm)

# Functions to pull data

In [7]:
nolyrics = 'nolyrics'
def getLyric(song, artist):
#return lyrics for a single song or None if not found
    try: 
        return api.search_song(song, artist).lyrics
    except:
        return nolyrics

In [8]:
#function...
def artist_album_collector(artistname):
    results=sp.search("artist:{}".format(artistname))
    artist_id = results['tracks']['items'][0]['artists'][0]['uri'].split(':')[2]
    return artist_id

# OutKast

In [9]:
#get artist data

results = sp.search("artist:OutKast")
artist_id = results['tracks']['items'][0]['artists'][0]['uri']
print(artist_id)

spotify:artist:1G9G7WwrXka3Z1r7aIDjI7


In [10]:
#get album details
albums = sp.artist_albums('1G9G7WwrXka3Z1r7aIDjI7', country="US", limit=50)
print(albums['next'])

#collect album details and print
album_ids = [album['uri'] for album in albums['items']]
print(len(album_ids))
print(album_ids[0])

all_tracks = []
for album_id in album_ids:
    tracks = sp.album_tracks(album_id, limit=50)
    all_tracks.append(tracks)
    
for tracks, album in zip(all_tracks, albums.get('items')):
    print(
        len(tracks.get('items')),
        "\t",
        album.get('name')
    )

#create df
track_ids = []
for tracks in all_tracks:
    album_tracks = []
    for track in tracks.get('items'):
        album_tracks.append(track.get('uri'))
    track_ids.append(album_tracks)


track_objects = []
for track_id_list in track_ids:
    tracks = sp.tracks(track_id_list)
    track_objects.append(tracks)

#collecting other features

audio_feature_objects = []
for track_id_list in track_ids:
    features = sp.audio_features(track_id_list)
    audio_feature_objects.append(features)

spotify_data = {
    "audio_features": audio_feature_objects,
    "tracks": track_objects
}

df = pd.DataFrame(columns=[
    'name',
    'duration_ms',
    'popularity',
    'num_markets',
    'album',
    'disc_number',
    'is_explicit',
    'track_number',
    'release_date',
    'artist',
    'danceability',
    'energy',
    'key',
    'loudness',
    'mode',
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
    'time_signature',
])
for album_info, album_features in zip(
        spotify_data.get('tracks'), 
        spotify_data.get('audio_features')
        ):
    for track_info, track_features in zip(
        album_info.get('tracks'),
        album_features
        ):
        y = {
            'name': track_info['name'],
            'duration_ms': track_info['duration_ms'],
            'popularity': track_info['popularity'],
            'num_markets': len(track_info['available_markets']),
            'album': track_info['album']['name'],
            'disc_number': track_info['disc_number'],
            'is_explicit': track_info['explicit'],
            'track_number': track_info['track_number'],
            'release_date': track_info['album']['release_date'],
            'artist': track_info['artists'][0]['name'],
            'danceability': track_features['danceability'],
            'energy': track_features['energy'],
            'key': track_features['key'],
            'loudness': track_features['loudness'],
            'mode': track_features['mode'],
            'speechiness': track_features['speechiness'],
            'acousticness': track_features['acousticness'],
            'instrumentalness': track_features['instrumentalness'],
            'liveness': track_features['liveness'],
            'valence': track_features['valence'],
            'tempo': track_features['tempo'],
            'time_signature': track_features['time_signature'],
        }
        df = df.append(y, ignore_index=True)
print(df.iloc[0])

https://api.spotify.com/v1/artists/1G9G7WwrXka3Z1r7aIDjI7/albums?offset=50&limit=50&include_groups=album,single,compilation,appears_on&market=US
50
spotify:album:3L6JBfRxhzGzhhRDlTEdBz
25 	 Idlewild
40 	 Speakerboxxx/The Love Below
24 	 Stankonia
16 	 Aquemini
15 	 ATLiens
17 	 Southernplayalisticadillacmuzik
1 	 Mighty "O" (Main Version)
4 	 The Way You Move / Hey Ya!
16 	 Big Boi & Dre Present, Outkast
32 	 90s Hip Hop
50 	 Hip Hop 100 Hits - Urban rap & R n B anthems inc. Jay Z, A$ap Rocky, Wu-Tang Clan & Nas
30 	 Throwback Thursday R&B Jams
30 	 New Year's Eve - NYE 2018/2019
38 	 Feel Good Friday
50 	 Coffeehouse Classics Vol. 1
50 	 Happiness Forever
18 	 "FRESH" The Album
15 	 Take Me To Rio (Ultimate Hits made in the iconic Sound of Brazil)
12 	 Street Swag, Vol. 2
12 	 Street Swag, Vol. 2
50 	 00s Party Mixtape
50 	 90 Sweet 90s Hits!
11 	 Best Of
27 	 The Essential UGK
50 	 R&B - 100 Hits - The Greatest R n B album - 100 R & B Classics featuring Usher, Pitbull and Justin Timb

In [ ]:
#add song lyrics to a list using genius api, then add this data to our spotify dataframe
df_list = []
for row in df['name']:

    df_list.append(getLyric(row, 'OutKast')),

dflyrics = pd.DataFrame(df_list, columns=['string_values'])

#add lyrics
df['lyrics'] = dflyrics['string_values']

Searching for "Intro - Main Version - Explicit" by OutKast...
Done.
Searching for "Mighty "O"" by OutKast...
Done.
Searching for "Peaches - Main Version - Explicit" by OutKast...
Done.
Searching for "Idlewild Blue (Don'tchu Worry 'Bout Me) - Main Version - Clean" by OutKast...
No results found for: 'Idlewild Blue (Don'tchu Worry 'Bout Me) - Main Version - Clean OutKast'
Searching for "Infatuation (Interlude) - Main Version - Explicit" by OutKast...
No results found for: 'Infatuation (Interlude) - Main Version - Explicit OutKast'
Searching for "N2U - Main Version - Explicit" by OutKast...
No results found for: 'N2U - Main Version - Explicit OutKast'
Searching for "Morris Brown - Main Version - Explicit" by OutKast...
No results found for: 'Morris Brown - Main Version - Explicit OutKast'
Searching for "Chronomentrophobia - Main Version - Explicit" by OutKast...
No results found for: 'Chronomentrophobia - Main Version - Explicit OutKast'
Searching for "The Train - Main Version - Explicit"

In [7]:
#cleaning
df = df[~df['lyrics'].isin(['nolyrics'])]
df = df[df['is_explicit'] == 1]
df=df.reset_index()
print(df)

     index                                              name duration_ms  \
0        0                   Intro - Main Version - Explicit      132026   
1        2                 Peaches - Main Version - Explicit      190813   
2        8               The Train - Main Version - Explicit      249853   
3        9  Life Is Like A Musical - Main Version - Explicit      134306   
4       11                                 Hollywood Divorce      323000   
5       13            Call The Law - Main Version - Explicit      291573   
6       16   Makes No Sense At All - Main Version - Explicit      173560   
7       17          In Your Dreams - Main Version - Explicit      214600   
8       20  Greatest Show On Earth - Main Version - Explicit      186826   
9       23           Dyin' To Live - Main Version - Explicit      127600   
10      24              A Bad Note - Main Version - Explicit      527200   
11      26                                      GhettoMusick      236720   
12      27  

In [8]:
#optional step to select the albums we want from this dataframe
outkast = df[df['album'].isin(['ATLiens', 'Stankonia', 'Aquemini', 'Southernplayalisticadillacmuzik', 'Idlewild', 'Speakerboxxx/The Love Below'])]

#add region info
region= []
for row in outkast['name']:
    region.append('south')

outkast['region'] = region
print(outkast)

    index                                              name duration_ms  \
0       0                   Intro - Main Version - Explicit      132026   
1       2                 Peaches - Main Version - Explicit      190813   
2       8               The Train - Main Version - Explicit      249853   
3       9  Life Is Like A Musical - Main Version - Explicit      134306   
4      11                                 Hollywood Divorce      323000   
5      13            Call The Law - Main Version - Explicit      291573   
6      16   Makes No Sense At All - Main Version - Explicit      173560   
7      17          In Your Dreams - Main Version - Explicit      214600   
8      20  Greatest Show On Earth - Main Version - Explicit      186826   
9      23           Dyin' To Live - Main Version - Explicit      127600   
10     24              A Bad Note - Main Version - Explicit      527200   
11     26                                      GhettoMusick      236720   
12     27                

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
#export the data for later use
outkast.to_csv (r'C:\Users\Sean\Documents\Thinkful\RAP NLP\CSV\outkastfinal.csv', index = None, header=True)